In [1]:
import random
from collections import Counter
from munkres import Munkres
niterations = 16000

# Check if this is the modified randomized version of the Munkres library,
# available at https://github.com/czlee/munkres/.
try:
    Munkres(random=True)
except TypeError:
    randomized_munkres = False
else:
    randomized_munkres = True

In [2]:
def hungarian_shuffle_first(costs):
    """Applies the Hungarian algorithm to `costs`, but permutes the rows and
    columns of the matrix first. Returns a list of column numbers."""
    n = len(costs)
    I = random.sample(range(n), n)
    J = random.sample(range(n), n)
    C = [[costs[i][j] for j in J] for i in I]
    m = Munkres()
    indices = m.compute(C)
    cols = [0] * n
    for i, j in indices:
        cols[I[i]] = J[j]
    return cols

def hungarian_shuffle_during(costs):
    """Applies a randomized version of the Hungarian algorithm.
    This only works with a modified version of the Munkres library, which
    can be cloned from https://github.com/czlee/munkres/.
    Randomizing during the algorithm seemed to have much the same effect as
    shuffling before it, so won't be pushing these changes into a pull
    request."""
    m = Munkres(random=True)
    indices = m.compute(costs)
    cols = [0] * len(costs)
    for i, j in indices:
        cols[i] = j
    return cols

def total_cost(costs, cols):
    """Computes the total cost according to `costs`, of the chosen columns
    `cols`."""
    rows = range(len(costs))
    return sum(costs[i][j] for i, j in zip(rows, cols))

def generate_bp_costs(d, k):
    """Generates an `4d`-by-`4d` cost matrix that assumes there have been
    `k` debates before now. (`d` represents the number of BP debates.)"""
    costs = []
    for i in range(4 * d):
        history = [random.choice(range(4)) for i in range(k)]
        counts = [history.count(pos) for pos in range(4)]
        row = counts * d  # replicate list `d` times
        costs.append(row)
    return costs
        
def run_simulations(costs, niterations, allocate):
    """Runs the Munkres-shuffle-first algorithm on the matrix `costs`
    `niterations` times, and returns a dict mapping solutions to
    frequencies (how many iterations that solution was obtained)."""
    results = Counter()
    for i in range(niterations):
        results[tuple(allocate(costs))] += 1
    return results

def verify_costs(costs, solutions):
    """Verifies that every key in solutions gives the same cost, and
    returns that cost if so."""
    total_costs = [total_cost(costs, solution) for solution in solutions]
    assert len(set(total_costs)) == 1
    return total_costs[0]

def frequencies_by_position(results):
    """Converts a sequence of solutions into just the positions (0 to 3,
    representing the 4 BP positions), and returns a dict mapping each such
    sequence to a list of frequencies for solutions that reduce to that 
    sequence of positions. Useful only for results of cost matrices generated 
    by `generate_bp_costs`."""
    reduced = {}
    for solution, frequency in results.items():
        positions = tuple(x % 4 for x in solution)
        frequencies = reduced.setdefault(positions, [])
        frequencies.append(frequency)
    return reduced

# British Parliamentary-like simulation

In [3]:
costs = generate_bp_costs(2, 3)
costs

[[0, 3, 0, 0, 0, 3, 0, 0],
 [1, 0, 1, 1, 1, 0, 1, 1],
 [2, 0, 1, 0, 2, 0, 1, 0],
 [1, 1, 0, 1, 1, 1, 0, 1],
 [2, 0, 0, 1, 2, 0, 0, 1],
 [1, 1, 1, 0, 1, 1, 1, 0],
 [1, 0, 2, 0, 1, 0, 2, 0],
 [1, 1, 0, 1, 1, 1, 0, 1]]

In [4]:
results = run_simulations(costs, niterations, hungarian_shuffle_first)
results

Counter({(0, 1, 3, 2, 5, 4, 7, 6): 141,
         (0, 1, 3, 2, 5, 7, 4, 6): 113,
         (0, 1, 3, 2, 6, 7, 5, 4): 117,
         (0, 1, 3, 4, 2, 7, 5, 6): 130,
         (0, 1, 3, 4, 6, 7, 5, 2): 114,
         (0, 1, 3, 6, 2, 7, 5, 4): 107,
         (0, 1, 3, 6, 5, 4, 7, 2): 116,
         (0, 1, 3, 6, 5, 7, 4, 2): 107,
         (0, 1, 5, 2, 6, 3, 7, 4): 124,
         (0, 1, 5, 2, 6, 7, 3, 4): 137,
         (0, 1, 5, 4, 2, 3, 7, 6): 120,
         (0, 1, 5, 4, 2, 7, 3, 6): 120,
         (0, 1, 5, 4, 6, 3, 7, 2): 123,
         (0, 1, 5, 4, 6, 7, 3, 2): 110,
         (0, 1, 5, 6, 2, 3, 7, 4): 114,
         (0, 1, 5, 6, 2, 7, 3, 4): 114,
         (0, 1, 7, 2, 5, 3, 4, 6): 125,
         (0, 1, 7, 2, 5, 4, 3, 6): 160,
         (0, 1, 7, 2, 6, 3, 5, 4): 123,
         (0, 1, 7, 4, 2, 3, 5, 6): 114,
         (0, 1, 7, 4, 6, 3, 5, 2): 127,
         (0, 1, 7, 6, 2, 3, 5, 4): 122,
         (0, 1, 7, 6, 5, 3, 4, 2): 129,
         (0, 1, 7, 6, 5, 4, 3, 2): 127,
         (0, 4, 1, 2, 5, 3, 7, 6): 137,


In [5]:
frequencies_by_position(results)

{(0, 0, 1, 2, 1, 3, 3, 2): [165,
  147,
  137,
  159,
  148,
  152,
  161,
  130,
  154,
  120,
  157,
  155,
  146,
  150,
  134,
  114],
 (0, 0, 3, 2, 1, 3, 1, 2): [99,
  126,
  128,
  127,
  123,
  124,
  145,
  125,
  122,
  109,
  120,
  97,
  119,
  108,
  125,
  109],
 (0, 1, 1, 0, 2, 3, 3, 2): [128,
  126,
  120,
  132,
  117,
  123,
  127,
  118,
  124,
  104,
  122,
  131,
  122,
  110,
  118,
  120],
 (0, 1, 1, 2, 2, 3, 3, 0): [110,
  110,
  113,
  128,
  140,
  119,
  120,
  140,
  137,
  114,
  118,
  142,
  113,
  124,
  114,
  111],
 (0, 1, 3, 0, 2, 3, 1, 2): [127,
  118,
  107,
  102,
  120,
  130,
  142,
  130,
  103,
  106,
  114,
  101,
  120,
  114,
  133,
  127],
 (0, 1, 3, 2, 1, 0, 3, 2): [160,
  159,
  127,
  143,
  153,
  133,
  141,
  116,
  162,
  153,
  160,
  144,
  150,
  141,
  142,
  139],
 (0, 1, 3, 2, 1, 3, 0, 2): [118,
  115,
  129,
  99,
  110,
  99,
  103,
  112,
  126,
  112,
  113,
  122,
  110,
  107,
  125,
  113],
 (0, 1, 3, 2, 2, 3, 1, 0): [99,

In [6]:
verify_costs(costs, results)

1

In [7]:
# Only run the next three cells if the randomized version of munkres is available
results = run_simulations(costs, niterations, hungarian_shuffle_during) if randomized_munkres else None

In [8]:
frequencies_by_position(results) if randomized_munkres else None

{(0, 0, 1, 2, 1, 3, 3, 2): [149,
  151,
  137,
  129,
  161,
  133,
  142,
  152,
  137,
  142,
  162,
  142,
  138,
  153,
  131,
  131],
 (0, 0, 3, 2, 1, 3, 1, 2): [96,
  102,
  109,
  121,
  118,
  115,
  98,
  115,
  111,
  112,
  111,
  107,
  120,
  121,
  103,
  109],
 (0, 1, 1, 0, 2, 3, 3, 2): [138,
  113,
  105,
  138,
  111,
  128,
  122,
  116,
  126,
  118,
  129,
  123,
  141,
  111,
  132,
  126],
 (0, 1, 1, 2, 2, 3, 3, 0): [154,
  118,
  102,
  123,
  110,
  133,
  134,
  141,
  122,
  129,
  141,
  133,
  110,
  118,
  139,
  139],
 (0, 1, 3, 0, 2, 3, 1, 2): [112,
  121,
  90,
  116,
  127,
  117,
  116,
  106,
  113,
  98,
  104,
  106,
  110,
  118,
  110,
  125],
 (0, 1, 3, 2, 1, 0, 3, 2): [144,
  130,
  123,
  119,
  138,
  115,
  128,
  122,
  126,
  129,
  121,
  133,
  126,
  124,
  138,
  123],
 (0, 1, 3, 2, 1, 3, 0, 2): [153,
  127,
  126,
  141,
  154,
  145,
  135,
  134,
  142,
  135,
  153,
  134,
  134,
  150,
  148,
  130],
 (0, 1, 3, 2, 2, 3, 1, 0): [126

In [9]:
verify_costs(costs, results) if randomized_munkres else None

1

# Random (Bernoulli i.i.d.) costs

In [10]:
def generate_random_costs(n, p):
    """Generates an `n`-by-`n` cost matrix in which each elements is
    i.i.d. Bernoulli(p). This is probably not a realistic cost matrix
    for a BP bracket."""
    return [[int(random.random() < p) for i in range(n)] for j in range(n)]

In [11]:
costs = generate_random_costs(8, 2/3)
costs

[[0, 1, 1, 1, 1, 0, 1, 1],
 [1, 1, 1, 0, 1, 0, 1, 1],
 [1, 1, 0, 0, 1, 1, 0, 1],
 [1, 1, 1, 1, 0, 1, 1, 0],
 [1, 1, 1, 1, 0, 1, 0, 1],
 [1, 1, 1, 1, 0, 1, 1, 0],
 [1, 0, 0, 0, 1, 1, 1, 0],
 [1, 0, 1, 1, 1, 1, 0, 1]]

In [12]:
results = run_simulations(costs, niterations, hungarian_shuffle_first)
results

Counter({(0, 5, 2, 4, 6, 7, 3, 1): 4208,
         (0, 5, 2, 7, 6, 4, 3, 1): 4179,
         (0, 5, 3, 4, 6, 7, 2, 1): 3787,
         (0, 5, 3, 7, 6, 4, 2, 1): 3826})

In [ ]:
verify_costs(costs, results)

0

In [ ]:
# Only run the next two cells if the randomized version of munkres is available
results = run_simulations(costs, niterations, hungarian_shuffle_during) if randomized_munkres else None
results

In [ ]:
verify_costs(costs, results) if randomized_munkres else None